# Complete API Testing - All Endpoints

This notebook provides a comprehensive overview and testing suite for all available API endpoints.

## API Structure:
1. **Authentication** - `/api/auth/`
2. **RBAC** - `/api/rbac/`
3. **Social Auth** - `/accounts/`
4. **Admin** - `/admin/`
5. **API Documentation** - `/api/schema/`

## Setup

In [1]:
import requests
import json
from pprint import pprint
from typing import Dict, Any

BASE_URL = "http://localhost:8000"
access_token = None

def print_response(response, title="Response"):
    """Pretty print response with title"""
    print(f"\n{'=' * 80}")
    print(f"{title}")
    print(f"{'=' * 80}")
    print(f"Status Code: {response.status_code}")
    print(f"\nResponse Body:")
    try:
        data = response.json()
        pprint(data)
    except:
        print(response.text)

def get_headers():
    """Get authorization headers"""
    if not access_token:
        return {}
    return {"Authorization": f"Bearer {access_token}"}

print("✅ Setup complete!")

✅ Setup complete!


## Get All Available Endpoints

Let's discover all endpoints using Django's URL patterns

In [2]:
# Get API schema
response = requests.get(f"{BASE_URL}/api/schema/")

if response.status_code == 200:
    schema = response.json()
    
    print("Available Endpoints:\n")
    print("=" * 80)
    
    if 'paths' in schema:
        for path, methods in sorted(schema['paths'].items()):
            print(f"\n📍 {path}")
            for method in methods.keys():
                if method.upper() in ['GET', 'POST', 'PUT', 'PATCH', 'DELETE']:
                    print(f"   {method.upper()}")
else:
    print("❌ Could not fetch API schema")
    print_response(response)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## Login First

Most endpoints require authentication

In [ ]:
login_data = {
    "email": "admin@replycompass.com",
    "password": "admin123"
}

response = requests.post(f"{BASE_URL}/api/auth/login/", json=login_data)

if response.status_code == 200:
    data = response.json()
    access_token = data.get('access')
    print("✅ Login successful!")
    print(f"\nUser: {data.get('user', {}).get('email')}")
    print(f"Access token: {access_token[:50]}...")
else:
    print("❌ Login failed!")
    print_response(response)

## 1. Authentication Endpoints Overview

In [ ]:
auth_endpoints = [
    ("GET", "/api/auth/user/", "Get current user"),
    ("POST", "/api/auth/registration/", "Register new user"),
    ("POST", "/api/auth/login/", "Login"),
    ("POST", "/api/auth/logout/", "Logout"),
    ("POST", "/api/auth/token/refresh/", "Refresh JWT token"),
    ("POST", "/api/auth/password/change/", "Change password"),
    ("POST", "/api/auth/password/reset/", "Request password reset"),
]

print("Authentication Endpoints:\n")
print("=" * 80)
for method, endpoint, description in auth_endpoints:
    print(f"{method:6} {endpoint:40} - {description}")

## 2. RBAC Endpoints Overview

In [ ]:
rbac_endpoints = [
    ("GET", "/api/rbac/roles/", "List all roles"),
    ("POST", "/api/rbac/roles/", "Create new role"),
    ("GET", "/api/rbac/roles/{id}/", "Get role details"),
    ("PUT", "/api/rbac/roles/{id}/", "Update role"),
    ("DELETE", "/api/rbac/roles/{id}/", "Delete role"),
    ("GET", "/api/rbac/permissions/", "List all permissions"),
    ("POST", "/api/rbac/permissions/", "Create permission"),
    ("GET", "/api/rbac/permissions/{id}/", "Get permission details"),
    ("GET", "/api/rbac/user-roles/", "List user role assignments"),
    ("POST", "/api/rbac/user-roles/", "Assign role to user"),
    ("GET", "/api/rbac/current-user/", "Get current user's RBAC info"),
]

print("RBAC Endpoints:\n")
print("=" * 80)
for method, endpoint, description in rbac_endpoints:
    print(f"{method:6} {endpoint:40} - {description}")

## 3. Test Current User Info

In [ ]:
response = requests.get(
    f"{BASE_URL}/api/auth/user/",
    headers=get_headers()
)
print_response(response, "Current User Info")

## 4. Test Current User's RBAC Info

In [ ]:
response = requests.get(
    f"{BASE_URL}/api/rbac/current-user/",
    headers=get_headers()
)
print_response(response, "Current User RBAC Info")

## 5. Complete System Overview

In [ ]:
print("\n" + "=" * 80)
print("COMPLETE SYSTEM OVERVIEW")
print("=" * 80)

# Get roles
roles_response = requests.get(f"{BASE_URL}/api/rbac/roles/", headers=get_headers())
if roles_response.status_code == 200:
    roles = roles_response.json()
    print(f"\n📋 Total Roles: {len(roles)}")
    for role in roles[:5]:
        print(f"   - {role['name']} (Level {role['level']})")

# Get permissions
perms_response = requests.get(f"{BASE_URL}/api/rbac/permissions/", headers=get_headers())
if perms_response.status_code == 200:
    permissions = perms_response.json()
    print(f"\n🔐 Total Permissions: {len(permissions)}")
    print(f"   Resources: {len(set(p['resource'] for p in permissions))}")

# Get user roles
user_roles_response = requests.get(f"{BASE_URL}/api/rbac/user-roles/", headers=get_headers())
if user_roles_response.status_code == 200:
    user_roles = user_roles_response.json()
    print(f"\n👥 Total User Role Assignments: {len(user_roles)}")

print("\n" + "=" * 80)

## 6. Test Filtering and Search

In [ ]:
# Test role filtering
print("Testing filters...\n")

# Filter by level
response = requests.get(
    f"{BASE_URL}/api/rbac/roles/?level__gte=50",
    headers=get_headers()
)
print(f"Roles with level >= 50: {len(response.json()) if response.status_code == 200 else 'Error'}")

# Search permissions
response = requests.get(
    f"{BASE_URL}/api/rbac/permissions/?search=user",
    headers=get_headers()
)
print(f"Permissions matching 'user': {len(response.json()) if response.status_code == 200 else 'Error'}")

# Filter by resource
response = requests.get(
    f"{BASE_URL}/api/rbac/permissions/?resource=admin",
    headers=get_headers()
)
print(f"Admin permissions: {len(response.json()) if response.status_code == 200 else 'Error'}")

## 7. Test Pagination

In [ ]:
# Test pagination on permissions endpoint
response = requests.get(
    f"{BASE_URL}/api/rbac/permissions/?page_size=5",
    headers=get_headers()
)

if response.status_code == 200:
    data = response.json()
    print(f"Results per page: {len(data)}")
    print(f"\nFirst 5 permissions:")
    for perm in data[:5]:
        print(f"  - {perm['name']}: {perm['description']}")
else:
    print_response(response)

## 8. Error Handling Tests

In [ ]:
print("Testing error scenarios...\n")

# Test 1: Access without authentication
response = requests.get(f"{BASE_URL}/api/rbac/roles/")
print(f"1. No auth header: Status {response.status_code}")

# Test 2: Invalid token
response = requests.get(
    f"{BASE_URL}/api/rbac/roles/",
    headers={"Authorization": "Bearer invalid_token"}
)
print(f"2. Invalid token: Status {response.status_code}")

# Test 3: Non-existent resource
response = requests.get(
    f"{BASE_URL}/api/rbac/roles/99999/",
    headers=get_headers()
)
print(f"3. Non-existent role: Status {response.status_code}")

# Test 4: Invalid data
response = requests.post(
    f"{BASE_URL}/api/rbac/roles/",
    json={"name": ""},  # Empty name
    headers=get_headers()
)
print(f"4. Invalid data: Status {response.status_code}")

## 9. API Documentation Links

In [ ]:
print("\n" + "=" * 80)
print("API DOCUMENTATION LINKS")
print("=" * 80)
print(f"\n📚 Swagger UI:     {BASE_URL}/api/schema/swagger/")
print(f"📚 ReDoc:          {BASE_URL}/api/schema/redoc/")
print(f"📚 OpenAPI Schema: {BASE_URL}/api/schema/")
print(f"🔧 Admin Panel:    {BASE_URL}/admin/")
print(f"\nOpen these URLs in your browser for interactive documentation!")
print("=" * 80)

## Summary

This notebook provides:
- ✅ Complete endpoint overview
- ✅ Authentication flow testing
- ✅ RBAC system testing
- ✅ Filter and search capabilities
- ✅ Error handling validation
- ✅ Pagination testing

For detailed testing of specific endpoints:
- Use `01_authentication_endpoints.ipynb` for auth
- Use `02_rbac_endpoints.ipynb` for RBAC
- Use `03_google_oauth.ipynb` for social auth